In [ ]:
import dgl.function as fn
import torch
import torch.nn as nn
import torch.nn.functional as F
from dgl.nn import SAGEConv
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE

model_path = os.path.join(input_folder,"gnn_model.pth")
loaded_model = GCN(in_feats, hidden_size, num_classes, conv_param)
loaded_model.load_state_dict(torch.load(model_path))

In [ ]:
# Extract node representations
with torch.no_grad():
    model.eval()
    node_embeddings = model(dgl_G, features).numpy()

# Reduce dimensions for plotting
tsne = TSNE(n_components=2)
node_embeddings_2d = tsne.fit_transform(node_embeddings)

# Plot node representations
plt.figure(figsize=(10, 8))
for i, label in enumerate(data['label'].unique()):
    indices = (data['label'] == label).values
    plt.scatter(node_embeddings_2d[indices, 0], node_embeddings_2d[indices, 1], label=label)
plt.legend()
plt.title('Node Representations (TSNE)')
plt.xlabel('TSNE Dimension 1')
plt.ylabel('TSNE Dimension 2')
plt.show()


In [ ]:
model_path = os.path.join(input_folder,"gnn_model_unsup.pth")
loaded_model = GCN(in_feats, hidden_size, num_classes, conv_param)
loaded_model.load_state_dict(torch.load(model_path))

In [ ]:
# Extract node representations
with torch.no_grad():
    model.eval()
    node_embeddings = model(dgl_G, features).numpy()

# Reduce dimensions for plotting
tsne = TSNE(n_components=2)
node_embeddings_2d = tsne.fit_transform(node_embeddings)

# Plot node representations
plt.figure(figsize=(10, 8))
for i, label in enumerate(data['label'].unique()):
    indices = (data['label'] == label).values
    plt.scatter(node_embeddings_2d[indices, 0], node_embeddings_2d[indices, 1], label=label)
plt.legend()
plt.title('Node Representations (TSNE)')
plt.xlabel('TSNE Dimension 1')
plt.ylabel('TSNE Dimension 2')
plt.show(